In [115]:
%%writefile MNIST.py
# Copyright 2015 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the 'License');
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an 'AS IS' BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
"""A simple MNIST classifier which displays summaries in TensorBoard.

 This is an unimpressive MNIST model, but it is a good example of using
tf.name_scope to make a graph legible in the TensorBoard graph explorer, and of
naming summary tags so that they are grouped meaningfully in TensorBoard.

It demonstrates the functionality of every TensorBoard dashboard.
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import numpy as np
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data

FLAGS = None


def train():
  # Import data
#   if FLAGS.data_loaded == 0:
  mnist = input_data.read_data_sets(FLAGS.data_dir,
                                    one_hot=True,
                                    fake_data=FLAGS.fake_data)

  sess = tf.InteractiveSession()

  # Create a multilayer model.

  # Input placeholders
  with tf.name_scope('input'):
    x = tf.placeholder(tf.float32, [None, 784], name='x-input')
    y_ = tf.placeholder(tf.float32, [None, 10], name='y-input')

  with tf.name_scope('input_reshape'):
    image_shaped_input = tf.reshape(x, [-1, 28, 28, 1])
    tf.image_summary('input', image_shaped_input, 10)

  # We can't initialize these variables to 0 - the network will get stuck.
  def weight_variable(shape):
    """Create a weight variable with appropriate initialization."""
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

  def bias_variable(shape):
    """Create a bias variable with appropriate initialization."""
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

  def variable_summaries(var, name):
    """Attach a lot of summaries to a Tensor."""
    with tf.name_scope('summaries'):
      tf.histogram_summary('raw'+name, var)
      
      mean = tf.reduce_mean(var)
      tf.scalar_summary('mean/' + name, mean)
      with tf.name_scope('stddev'):
        stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
      tf.scalar_summary('stddev/' + name, stddev)
      tf.scalar_summary('max/' + name, tf.reduce_max(var))
      tf.scalar_summary('min/' + name, tf.reduce_min(var))
      tf.histogram_summary(name, var)

  def nn_layer(input_tensor, input_dim, output_dim, layer_name, act=tf.nn.relu):
    """Reusable code for making a simple neural net layer.

    It does a matrix multiply, bias add, and then uses relu to nonlinearize.
    It also sets up name scoping so that the resultant graph is easy to read,
    and adds a number of summary ops.
    """
    # Adding a name scope ensures logical grouping of the layers in the graph.
    with tf.name_scope(layer_name):
      # This Variable will hold the state of the weights for the layer
      with tf.name_scope('weights'):
        weights = weight_variable([input_dim, output_dim])
        variable_summaries(weights, layer_name + '/weights')
      with tf.name_scope('biases'):
        biases = bias_variable([output_dim])
        variable_summaries(biases, layer_name + '/biases')
      with tf.name_scope('Wx_plus_b'):
        preactivate = tf.matmul(input_tensor, weights) + biases
        tf.histogram_summary(layer_name + '/pre_activations', preactivate)
      activations = act(preactivate, name='activation')
      tf.histogram_summary(layer_name + '/activations', activations)
      return (activations, weights)

  hidden1, weights = nn_layer(x, 784, 500, 'layer1')

  with tf.name_scope('dropout'):
    keep_prob = tf.placeholder(tf.float32)
    tf.scalar_summary('dropout_keep_probability', keep_prob)
    dropped = tf.nn.dropout(hidden1, keep_prob)

  # Do not apply softmax activation yet, see below.
  y, weights = nn_layer(dropped, 500, 10, 'layer2', act=tf.identity)

  with tf.name_scope('cross_entropy'):
    # The raw formulation of cross-entropy,
    #
    # tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(tf.softmax(y)),
    #                               reduction_indices=[1]))
    #
    # can be numerically unstable.
    #
    # So here we use tf.nn.softmax_cross_entropy_with_logits on the
    # raw outputs of the nn_layer above, and then average across
    # the batch.
    diff = tf.nn.softmax_cross_entropy_with_logits(y, y_)
    with tf.name_scope('total'):
      cross_entropy = tf.reduce_mean(diff)
    tf.scalar_summary('cross entropy', cross_entropy)

  with tf.name_scope('train'):
    train_step = tf.train.AdamOptimizer(FLAGS.learning_rate).minimize(
        cross_entropy)

  with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
      correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    with tf.name_scope('accuracy'):
      accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.scalar_summary('accuracy', accuracy)

  # Merge all the summaries and write them out to /tmp/mnist_logs (by default)
  merged = tf.merge_all_summaries()
  train_writer = tf.train.SummaryWriter(FLAGS.summaries_dir + '/train',
                                        sess.graph)
  test_writer = tf.train.SummaryWriter(FLAGS.summaries_dir + '/test')
  tf.initialize_all_variables().run()

  # Train the model, and also write summaries.
  # Every 10th step, measure test-set accuracy, and write test summaries
  # All other steps, run train_step on training data, & add training summaries

  def feed_dict(train):
    """Make a TensorFlow feed_dict: maps data onto Tensor placeholders."""
    if train or FLAGS.fake_data:
      xs, ys = mnist.train.next_batch(100, fake_data=FLAGS.fake_data)
      k = FLAGS.dropout
    else:
      xs, ys = mnist.test.images, mnist.test.labels
      k = 1.0
    return {x: xs, y_: ys, keep_prob: k}

  _weights_ = []
  for i in range(FLAGS.max_steps):
    _w_ = sess.run(weights,feed_dict=feed_dict(True))
    _weights_.append(_w_)
    
    if i % 10 == 0:  # Record summaries and test-set accuracy
      summary, acc = sess.run([merged, accuracy], feed_dict=feed_dict(False))
      test_writer.add_summary(summary, i)
      print('Accuracy at step %s: %s' % (i, acc))
    else:  # Record train set summaries, and train
      if i % 100 == 99:  # Record execution stats
        run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
        run_metadata = tf.RunMetadata()
        summary, _ = sess.run([merged, train_step],
                              feed_dict=feed_dict(True),
                              options=run_options,
                              run_metadata=run_metadata)
        train_writer.add_run_metadata(run_metadata, 'step%03d' % i)
        train_writer.add_summary(summary, i)
        print('Adding run metadata for', i)
      else:  # Record a summary
        summary, _ = sess.run([merged, train_step], feed_dict=feed_dict(True))
        train_writer.add_summary(summary, i)
  train_writer.close()
  test_writer.close()
  print("weights", len(_weights_),len(_weights_[0][0]))

def main(_):
  if tf.gfile.Exists(FLAGS.summaries_dir):
    tf.gfile.DeleteRecursively(FLAGS.summaries_dir)
  tf.gfile.MakeDirs(FLAGS.summaries_dir)
  train()


if __name__ == '__main__':
  parser = argparse.ArgumentParser()
  parser.add_argument('--data_loaded', type=int,
                      default=0,
                      help='If true, the data was already loaded.')
  parser.add_argument('--fake_data', nargs='?', const=True, type=bool,
                      default=False,
                      help='If true, uses fake data for unit testing.')
  parser.add_argument('--max_steps', type=int, default=1000,
                      help='Number of steps to run trainer.')
  parser.add_argument('--learning_rate', type=float, default=0.001,
                      help='Initial learning rate')
  parser.add_argument('--dropout', type=float, default=0.9,
                      help='Keep probability for training dropout.')
  parser.add_argument('--data_dir', type=str, default='/tmp/data',
                      help='Directory for storing data')
  parser.add_argument('--summaries_dir', type=str, default='/tmp/mnist_logs',
                      help='Summaries directory')
  FLAGS = parser.parse_args()
  tf.app.run()

Overwriting MNIST.py


In [116]:
!python MNIST.py --max_steps 100

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Accuracy at step 0: 0.0827
Accuracy at step 10: 0.7124
Accuracy at step 20: 0.8176
Accuracy at step 30: 0.8516
Accuracy at step 40: 0.8686
Accuracy at step 50: 0.8858
Accuracy at step 60: 0.8883
Accuracy at step 70: 0.9007
Accuracy at step 80: 0.9036
Accuracy at step 90: 0.912
Adding run metadata for 99
weights 100 10


In [94]:
!rm -r /tmp/serialized-mnist
!python serialize.py --logdir /tmp/mnist_logs --target /tmp/serialized-mnist

About to load Multiplexer. This may take some time.
Multiplexer load finished. Starting TensorBoard server.
Server setup! Downloading data from the server.
127.0.0.1 - - [10/Nov/2016 22:59:32] "GET /data/runs HTTP/1.1" 200 -
127.0.0.1 - - [10/Nov/2016 22:59:32] "GET /data/scalars HTTP/1.1" 200 -
127.0.0.1 - - [10/Nov/2016 22:59:32] "GET /data/scalars?run=test&tag=min%2Flayer2%2Fweights HTTP/1.1" 200 -
127.0.0.1 - - [10/Nov/2016 22:59:32] "GET /data/scalars?run=test&tag=mean%2Flayer2%2Fweights HTTP/1.1" 200 -
127.0.0.1 - - [10/Nov/2016 22:59:32] "GET /data/scalars?run=test&tag=mean%2Flayer1%2Fweights HTTP/1.1" 200 -
127.0.0.1 - - [10/Nov/2016 22:59:32] "GET /data/scalars?run=test&tag=min%2Flayer1%2Fbiases HTTP/1.1" 200 -
127.0.0.1 - - [10/Nov/2016 22:59:32] "GET /data/scalars?run=test&tag=cross+entropy HTTP/1.1" 200 -
127.0.0.1 - - [10/Nov/2016 22:59:32] "GET /data/scalars?run=test&tag=stddev%2Flayer1%2Fweights HTTP/1.1" 200 -
127.0.0.1 - - [10/Nov/2016 22:59:32] "GET /data/scalars?run=

In [95]:
!ls /tmp/serialized-mnist/

compressedHistograms_run_test_tag_layer1_2Factivations.json
compressedHistograms_run_test_tag_layer1_2Fbiases.json
compressedHistograms_run_test_tag_layer1_2Fpre_activations.json
compressedHistograms_run_test_tag_layer1_2Fweights.json
compressedHistograms_run_test_tag_layer2_2Factivations.json
compressedHistograms_run_test_tag_layer2_2Fbiases.json
compressedHistograms_run_test_tag_layer2_2Fpre_activations.json
compressedHistograms_run_test_tag_layer2_2Fweights.json
compressedHistograms_run_test_tag_rawlayer1_2Fbiases.json
compressedHistograms_run_test_tag_rawlayer1_2Fweights.json
compressedHistograms_run_test_tag_rawlayer2_2Fbiases.json
compressedHistograms_run_test_tag_rawlayer2_2Fweights.json
compressedHistograms_run_train_tag_layer1_2Factivations.json
compressedHistograms_run_train_tag_layer1_2Fbiases.json
compressedHistograms_run_train_tag_layer1_2Fpre_activations.json
compressedHistograms_run_train_tag_layer1_2Fweights.json
compressedHistograms_run_train_tag_layer2_2Factivations.j

In [98]:
!cat  /tmp/serialized-mnist/compressedHistograms_run_train_tag_rawlayer1_2Fweights.json

[[1478836751.780183, 1, [[0, -0.20098741352558136], [668, -0.13602662535491533], [1587, -0.093811976423785456], [3085, -0.047433053684848937], [5000, 0.00035885736690291416], [6915, 0.04762725248416131], [8413, 0.094229408964759995], [9332, 0.13681043462576623], [10000, 0.20099811255931643]]], [1478836751.810258, 2, [[0, -0.20188668370246887], [668, -0.13608042050870534], [1587, -0.093871537773587213], [3085, -0.047479530327565099], [5000, 0.00030783898383663906], [6915, 0.047560044006812796], [8413, 0.094189862760638135], [9332, 0.13676464324694329], [10000, 0.20192053914070016]]], [1478836751.84117, 3, [[0, -0.2028348296880722], [668, -0.13611562165903135], [1587, -0.093909251990938289], [3085, -0.047507962777323193], [5000, 0.00027989391736744874], [6915, 0.047517651101875245], [8413, 0.094139599808006208], [9332, 0.13673162375478323], [10000, 0.20289084315299447]]], [1478836751.867335, 4, [[0, -0.20375797152519226], [668, -0.13612186282592387], [1587, -0.09394423020465742], [3085, 

In [14]:
# %tb
print(tf.__version__);

0.11.0
